# Previsão do Cargo Ideal

## 1. Introdução
Neste projeto, construirei um modelo de aprendizado de máquina para prever o cargo ideal na área de dados com base no estudo *State of Data Brazil 2022*. O problema é abordado como uma tarefa de classificação supervisionada, onde o modelo aprenderá a partir de dados rotulados para determinar a posição mais adequada para um candidato com base em diversos atributos.  

Utilizarei o seguinte pipeline baseado no processo KDD (Knowledge Discovery in Databases):  

1. **Seleção** – Identificar e selecionar as instâncias e atributos mais relevantes para a análise.  
2. **Pré-processamento** – Tratar dados ausentes, incorretos ou inconsistentes para melhorar a qualidade geral.  
3. **Transformação** – Converter os dados para um formato compatível com as técnicas de modelagem escolhidas.  
4. **Mineração de Dados** – Treinar e comparar modelos de classificação para prever o cargo ideal.  
5. **Avaliação** – Avaliar o desempenho dos modelos e analisar os resultados para selecionar a melhor abordagem.  

Neste notebook, realizarei a análise exploratória dos dados (EDA), cobrindo as etapas 1 a 4 do pipeline acima. O principal objetivo é extrair insights que revelem padrões entre diferentes perfis de candidatos a partir das variáveis disponíveis. Mesmo antes de construir o modelo final, essa análise fornecerá informações úteis sobre tendências de adequação a cargos na área de dados. Além disso, explicarei cada etapa em detalhes, justificando as decisões tomadas ao longo do processo.

## 2. Configurações Iniciais 

### 2.1 Importando as Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, ConfusionMatrixDisplay, confusion_matrix

## 2.2 Compeendendo a Base da Dados
- O dataset foi coletado do kaggle: https://www.kaggle.com/datasets/datahackers/state-of-data-2022
- Mais detalhes no site da pesquisa: https://www.stateofdata.com.br
- Os dados da pesquisa foram segmentados em oito partes, sendo as mais adequadas para nossas previsões as partes 1, 2 e 4, que correspondem, respectivamente, 
    - Dados demográficos, 
    - Dados sobre a carreira  
    - Conhecimentos na área de dados.

In [4]:
# Importando a base de dados
df = pd.read_csv('../base/State_of_data_2022.csv')

# Função para filtrar colunas
def filtrar_colunas(df):
    colunas_desejadas = [
        "('P1_a ', 'Idade')", "('P1_b ', 'Genero')", "('P1_c ', 'Cor/raca/etnia')", "('P1_d ', 'PCD')", "('P1_g ', 'vive_no_brasil')", "('P1_i ', 'Estado onde mora')",
        "('P1_l ', 'Nivel de Ensino')", "('P1_m ', 'Área de Formação')", "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')", "('P4_f ', 'Entre as linguagens listadas abaixo, qual é a sua preferida?')",
        "('P4_g ', 'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?')", "('P4_h ', 'Dentre as opções listadas, qual sua Cloud preferida?')", "('P2_f ', 'Cargo Atual')"
    ]
    return df[colunas_desejadas]

# Aplicando a função
df = filtrar_colunas(df)

# Renomeando as colunas
mapeamento_colunas = {
    "('P1_a ', 'Idade')": 'idade',
    "('P1_b ', 'Genero')": 'genero',
    "('P1_c ', 'Cor/raca/etnia')": 'etnia',
    "('P1_d ', 'PCD')": 'pcd',
    "('P1_g ', 'vive_no_brasil')": 'vive_no_brasil',
    "('P1_i ', 'Estado onde mora')": 'estado_moradia',
    "('P1_l ', 'Nivel de Ensino')": 'nivel_ensino',
    "('P1_m ', 'Área de Formação')": 'formacao',
    "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')": 'tempo_experiencia_dados',
    "('P4_f ', 'Entre as linguagens listadas abaixo, qual é a sua preferida?')": 'linguagens_preferidas',
    "('P4_g ', 'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?')": 'bancos_de_dados',
    "('P4_h ', 'Dentre as opções listadas, qual sua Cloud preferida?')": 'cloud_preferida',
    "('P2_f ', 'Cargo Atual')": 'cargo'
}

df.rename(columns=mapeamento_colunas, inplace=True)
df.head()

,idade,genero,etnia,pcd,vive_no_brasil,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
0,39.0,Masculino,Parda,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,R,"PostgreSQL, Oracle, MySQL, SQL SERVER",Amazon Web Services (AWS),DBA/Administrador de Banco de Dados
1,32.0,Masculino,Parda,Não,True,Pará (PA),Graduação/Bacharelado,Outras Engenharias,de 3 a 4 anos,NaN,NaN,NaN,NaN
2,53.0,Masculino,Branca,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,Python,"PostgreSQL, MySQL, Oracle, DB2",Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
3,27.0,Masculino,Branca,Não,True,Minas Gerais (MG),Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...,de 4 a 6 anos,Python,Google BigQuery,Não sei opinar,Cientista de Dados/Data Scientist
4,46.0,Feminino,Branca,Não,True,Pará (PA),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Não tenho experiência na área de dados,Python,Microsoft Access,Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4271 entries, 0 to 4270
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   idade                    4197 non-null   float64
 1   genero                   4262 non-null   object 
 2   etnia                    4271 non-null   object 
 3   pcd                      4271 non-null   object 
 4   vive_no_brasil           4271 non-null   bool   
 5   estado_moradia           4156 non-null   object 
 6   nivel_ensino             4271 non-null   object 
 7   formacao                 4149 non-null   object 
 8   tempo_experiencia_dados  3694 non-null   object 
 9   linguagens_preferidas    2979 non-null   object 
 10  bancos_de_dados          2978 non-null   object 
 11  cloud_preferida          2981 non-null   object 
 12  cargo                    2981 non-null   object 
dtypes: bool(1), float64(1), object(11)
memory usage: 404.7+ KB


In [8]:
# Verificando os valores nulos 
df.isna().sum()

idade                        74
genero                        9
etnia                         0
pcd                           0
vive_no_brasil                0
estado_moradia              115
nivel_ensino                  0
formacao                    122
tempo_experiencia_dados     577
linguagens_preferidas      1292
bancos_de_dados            1293
cloud_preferida            1290
cargo                      1290
dtype: int64

In [9]:
df.duplicated().sum()

53